In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [2]:
import pandas as pd
from google.oauth2 import service_account

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import ztest
from statsmodels.stats import weightstats
from datetime import datetime
import statsmodels.stats.weightstats as ws
import math 

import plotly.io as pio
import plotly.graph_objects as go
import plotly as py
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf



pio.renderers.default='notebook' # для отображения графиков плотли в nbviewer
init_notebook_mode(connected = True) # для работы с плотли оффлайн
cf.go_offline()

pd.set_option('display.max_colwidth', 150)
pd.set_option('display.max_columns', None)


In [17]:
# sql запрос для получения данных о кликах по каталогу 
def categ_sql():
    credentials = service_account.Credentials.from_service_account_file(
    '/Users/lxndrarura/Python Santehnika/key/spry-compound-139714-3c5694cec4ca.json')
    project_id = 'spry-compound-139714' 
   
    query = '''
    select 
        *
    from `spry-compound-139714.SNTO_lyapakhina.head_catalog_events11`
    '''
    ct = pd.read_gbq(query, project_id=project_id, credentials=credentials)
    return ct

# sql запрос для получения данных о кликах по шапке каталога 
def bar_sql():
    credentials = service_account.Credentials.from_service_account_file(
    '/Users/lxndrarura/Python Santehnika/key/spry-compound-139714-3c5694cec4ca.json')
    project_id = 'spry-compound-139714' 
   
    query = '''
    select 
        *
    from `spry-compound-139714.SNTO_lyapakhina.head_bar_events11`
    '''
    bar = pd.read_gbq(query, project_id=project_id, credentials=credentials)
    return bar

# sql запрос для получения данных о сессиях
def sessions_sql():
    credentials = service_account.Credentials.from_service_account_file(
    '/Users/lxndrarura/Python Santehnika/key/spry-compound-139714-3c5694cec4ca.json')
    project_id = 'spry-compound-139714' 
   
    query = '''
    select 
        *
    from `spry-compound-139714.SNTO_lyapakhina.head_sessions_count11`
    '''
    ses = pd.read_gbq(query, project_id=project_id, credentials=credentials)
    return ses

# sql запрос для получения данных о сессиях с отказом после открытия каталога
def rejections_sql():
    credentials = service_account.Credentials.from_service_account_file(
    '/Users/lxndrarura/Python Santehnika/key/spry-compound-139714-3c5694cec4ca.json')
    project_id = 'spry-compound-139714' 
   
    query = '''
    select 
        *
    from `spry-compound-139714.SNTO_lyapakhina.head_sessions_with_rejection111`
    '''

    reg = pd.read_gbq(query, project_id=project_id, credentials=credentials)
    return reg


# получение таблицы с информацией о распределении кликов по каталогу 
def to_categ(ct):
    categ = ct[(ct['eventAction']=='header_nav_catalog_click')][['hitId', 'eventLabel']]
    categ = categ[categ['eventLabel'].str.contains("/") == True]
    categ['eventLabel'] = categ['eventLabel'].apply(lambda x: x.lower())
    categ = categ[(categ['eventLabel'].str.contains("каталог товаров") == True) | (categ['eventLabel'].str.contains("каталог") == True)]
    categ['one'] = categ['eventLabel'].apply(lambda x: x[x.index('/', 0)+1:])
    categ['one'] = categ['one'].apply(lambda x: x[:x.index('/', 0)] if ("/" in x) == True else x)
    categ['last'] = categ['eventLabel'].apply(lambda x: x[x.rindex('/', 0)+1:])
    categ = categ[['one', 'last', 'hitId']]
    categ = categ.groupby(['one', 'last'])['hitId'].count().reset_index().rename(columns = {'hitId': 'count_sum'})    
    return categ

# визуализация распределения кликов по каталогу общая
def categ_distribution_visualization_all(categ):
    # визуализация 
    df = categ
    fig = px.treemap(df, path=[px.Constant("all"), 'one', 'last'], values='count_sum',
    #                   color='count_sum', 
    #                  hover_data=['count_sum'],
                      color_continuous_scale='RdBu',
                     color_continuous_midpoint=np.average(df['count_sum'], weights=df['count_sum'])
                    )
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    return fig.show()

# визуализация распределения кликов по каталогу верхний уровень вложенности
def categ_distribution_visualization_upper(categ):
    df = categ.groupby('one')['count_sum'].sum().reset_index()
    fig = px.treemap(df, path=[px.Constant("all"), 'one'], values='count_sum',
                  color='count_sum', hover_data=['count_sum'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['count_sum'], weights=df['count_sum'])
                )
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    return fig.show()

# визуализация распределения кликов по каталогу верхнидний уровень вложенности
def categ_distribution_visualization_lower(categ):
    df = categ.groupby('last')['count_sum'].sum().reset_index()
    fig = px.treemap(df, path=[px.Constant("all"), 'last'], values='count_sum',
                  color='count_sum', hover_data=['count_sum'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df['count_sum'], weights=df['count_sum'])
                )
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    return fig.show()

# получение таблицы с метриками по каталогу
def result_catalog(ct, ses, reg):
    res = ses.groupby('f0_')['sessions_count'].sum().reset_index()
    sessions_sum = res['sessions_count'].sum()
    res['GT%'] = (100/sessions_sum)*res['sessions_count']
    res = res.merge(ct.groupby('f0_')['sessionId'].nunique().reset_index().rename(columns = {'sessionId':'catalog_open_sessions'}), on='f0_', how='left')
    res['CR в уник. клики,%'] = (res['catalog_open_sessions']/res['sessions_count'])*100
    res = res.merge(ct[ct['eventLabel']=='open'].groupby('f0_')['hitId'].count().reset_index().rename(columns = {'hitId':'catalog_open_count'}), on='f0_', how='left')
    res['Частота кликов'] = res['catalog_open_count']/res['catalog_open_sessions']
    res = res.merge(reg.groupby('f0_')['sessionId'].nunique().reset_index().rename(columns = {'sessionId': 'Сессии с выходом'}), on='f0_', how='left')
    res['Показатель выхода,%'] = (res['Сессии с выходом']/res['catalog_open_sessions'])*100
    res = res.round(2)
    r = res.T
    r.columns = r.iloc[0]
    r = r[1:]
    r = r[['before', 'after']]
    r['diff,%'] = (100/r['before'])*r['after']-100
    r['diff,%'] = r['diff,%'].apply(lambda x: round(x, 2))
    return r 

# проверка на стат значимость частоты кликов по каталогу
def stat_significance_catalog_freq(ct):
    
    cc = ct[ct['eventLabel']=='open'].groupby(['sessionId', 'f0_'])['hitId'].count().reset_index()
    a = cc[cc['f0_']=='after']
    b = cc[cc['f0_']=='before']
    
    if a['hitId'].mean() > b['hitId'].mean():
        t_result_bill = weightstats.ttest_ind(a['hitId'], b['hitId'],
                         usevar='unequal', alternative='larger')
        if t_result_bill[1] <= 0.05:
            print('Частота кликов по каталогу в новой версии больше чем в старой, разница стат.значима на уровне значимости a = 5%, p_value =', '{:.19f}'.format(t_result_bill[1]))
        else:
            print('Частота кликов по каталогу в новой версии не больше чем в старой, разница не стат.значима на уровне значимости a = 5%, p_value =', '{:.19f}'.format(t_result_bill[1]))           

            #     print('Ttest_indResult', t_result_bill[0], '{:.19f}'.format(t_result_bill[1]))

# визуализация таблицы с метриками по каталогу
def result_catalog_visualisation(r): 
    columns_list = r.columns
    columns_list = columns_list.insert(0, '') 
    columns_list

    fig = go.Figure(data=[go.Table(
     # columnorder = [1,4],
      columnwidth = [150,160,160,160],
      header = dict(
        values = columns_list,
        line_color='darkslategray',
        fill_color='rgb(103,112,244)',
        align=['left','center'],
        font=dict(color='white', size=12),
        height=40
      ),
      cells=dict(
        values=[r.index, r['before'], r['after'], r['diff,%']],
        line_color='darkslategray',
        fill=dict(color=[['rgb(229, 238, 255)','rgb(229, 238, 255)','rgb(229, 238, 255)', '#a1bcf7','rgb(229, 238, 255)','#a1bcf7','rgb(229, 238, 255)','#a1bcf7'], ['white','white','white', '#d9d9d9','white','#d9d9d9','white','#d9d9d9'], ['white','white','white', '#d9d9d9','white','#d9d9d9','white','#d9d9d9'], ['white','white','white', '#d9d9d9','white','#d9d9d9','white','#d9d9d9']]),
        align=['left', 'center'],
        font_size=12,
        height=30)
        )
    ])
    return fig.show()

# получение таблицы метрик по шапке
def result_bar(bar, ses):
    res2 = ses.groupby('f0_')['sessions_count'].sum().reset_index()
    sessions_sum = res2['sessions_count'].sum()
    res2['GT%'] = (100/sessions_sum)*res2['sessions_count']
    
    res2 = res2.merge(bar.groupby('f0_')['sessionId'].nunique().reset_index().rename(columns = {'sessionId':'bar_click_sessions'}), on='f0_', how='left')
    for i in bar['eventLabel'].unique():
        res2 = res2.merge(bar[bar['eventLabel']==i].groupby('f0_')['sessionId'].nunique().reset_index().rename(columns = {'sessionId':'bar_click_sessions_{}'.format(i)}), on='f0_', how='left')
   
    res2['%CR в уник. клики'] = (res2['bar_click_sessions']/res2['sessions_count'])*100
    for i in bar['eventLabel'].unique():
        res2['%CR в уник. клики_{}'.format(i)] = (res2['bar_click_sessions_{}'.format(i)]/res2['sessions_count'])*100

    res2 = res2.merge(bar.groupby('f0_')['open_count'].sum().reset_index().rename(columns = {'open_count':'Клики'}), on='f0_', how='left')
    for i in bar['eventLabel'].unique():
        res2 = res2.merge(bar[bar['eventLabel']==i].groupby('f0_')['open_count'].sum().reset_index().rename(columns = {'open_count':'Клики_{}'.format(i)}), on='f0_', how='left')

    res2['Частота кликов'] = (res2['Клики']/res2['bar_click_sessions'])
    for i in bar['eventLabel'].unique():
        res2['Частота кликов_{}'.format(i)] = res2['Клики_{}'.format(i)]/res2['bar_click_sessions_{}'.format(i)]

    res2['%GT_Клики'] = (res2['Клики']/res2['Клики'])*100
    for i in bar['eventLabel'].unique():
        res2['%GT_Клики_{}'.format(i)] = (res2['Клики_{}'.format(i)]/res2['Клики'])*100
    
    res2 = res2.round(2)
    r2 = res2.T
    r2.columns = r2.iloc[0]
    r2 = r2[1:]
    r2 = r2[['before', 'after']]
    r2['diff,%'] = (100/r2['before'])*r2['after']-100
    r2['diff,%'] = r2['diff,%'].apply(lambda x: round(x, 2))
    return r2

# визуализация таблицы с метриками по шапке 
def result_bar_visualization(r2):
    columns_list = r2.columns
    columns_list = columns_list.insert(0, '') 

    fig = go.Figure(data=[go.Table(
     # columnorder = [1,4],
      columnwidth = [210,160,160,160],
      header = dict(
        values = columns_list,
        line_color='darkslategray',
        fill_color='rgb(103,112,244)',
        align=['left','center'],
        font=dict(color='white', size=12),
        height=40
      ),
      cells=dict(
        values=[r2.index, r2['before'], r2['after'], r2['diff,%']],
        line_color='darkslategray',
        fill=dict(color=[['#b8cdf9', '#b8cdf9', '#b8cdf9', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)','#b8cdf9', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', '#b8cdf9', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', '#b8cdf9', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', '#b8cdf9', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)', 'rgb(229, 238, 255)' ], 
                         ['#f2f2f2', '#f2f2f2', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white','#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white','#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', ], 
                         ['#f2f2f2', '#f2f2f2', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white','#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white','#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', ], 
                         ['#f2f2f2', '#f2f2f2', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', '#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white','#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white','#f2f2f2', 'white', 'white', 'white', 'white', 'white', 'white', ]
                        ]),
        align=['left', 'center'],
        font_size=12,
        height=30)
        )
    ])
    return fig.show()

# визуализация распределения кликов по элементам шапки 
def bar_gt_b(bar):
    b1 = bar[bar['f0_']=='before'].groupby('eventLabel')['open_count'].sum().reset_index()
    b2 = bar[bar['f0_']=='after'].groupby('eventLabel')['open_count'].sum().reset_index()
    
    
    pyplt=py.offline.plot
    labels=b1['eventLabel']
    values=b1['open_count']
    trace=[go.Pie(
        labels=labels,
        values=values,
        hole=0.7,   # Размер заготовки центра кольца управления
        hoverinfo='label+percent'   
    )]
    layout=go.Layout(
        title='before. Клики по элементам шапки'
    )
    fig=go.Figure(data=trace,layout=layout)
    return fig.show()

def bar_gt_a(bar):
    b1 = bar[bar['f0_']=='before'].groupby('eventLabel')['open_count'].sum().reset_index()
    b2 = bar[bar['f0_']=='after'].groupby('eventLabel')['open_count'].sum().reset_index()
    
    
    pyplt=py.offline.plot
    labels=b2['eventLabel']
    values=b2['open_count']
    trace=[go.Pie(
        labels=labels,
        values=values,
        hole=0.7,   # Размер заготовки центра кольца управления
        hoverinfo='label+percent'   
    )]
    layout=go.Layout(
        title='after. Клики по элементам шапки'
    )
    fig=go.Figure(data=trace,layout=layout)
    return fig.show()
    

In [10]:
# создание таблиц с помощью sql запроса 
ct = categ_sql()
bar = bar_sql()
ses = sessions_sql()
reg = rejections_sql()


# Вовлеченность в элементы хедера и каталога после редизайна             

- <b>before</b>: 2021.11-17 - 2021.11.23
- <b>after</b>: 2021.11.29 - 2021.12.04
- <b>Срок внедрения</b>: 2021-11-24
- <b>desktop </b>     


##  Каталог. Распределение кликов по категориям


In [11]:
categ = to_categ(ct)
categ_distribution_visualization_all(categ)


##  Каталог. Распределение кликов по категориям верхнего уровня

In [12]:
categ_distribution_visualization_upper(categ)



## Каталог. Распределение кликов по категориям нижнего уровня


In [7]:
categ_distribution_visualization_lower(categ)


##  Каталог. Сравнение показателей вовлеченности 


In [13]:
r = result_catalog(ct, ses, reg)
result_catalog_visualisation(r)


In [19]:
# stat_significance_catalog_freq(ct)


##  Шапка. Сравнение показателей вовлеченности 


In [15]:
r2 = result_bar(bar, ses)
result_bar_visualization(r2)




## Шапка. Распределение кликов по элементам до и после


In [18]:
bar_gt_b(bar)
bar_gt_a(bar)
